In [1]:
import guppylang
from guppylang import guppy
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit, measure, h, cx
guppylang.enable_experimental_features()


### Variable not defined

In [2]:
@guppy
def bad(b: bool) -> int:
    if b:
        x = 4
    return x  # x not defined if b is False

bad.compile()  # Raises an error

Error: Variable not defined (at <In[2]>:5:11)
  | 
3 |     if b:
4 |         x = 4
5 |     return x  # x not defined if b is False
  |            ^ `x` might be undefined ...

Note:
  | 
2 | def bad(b: bool) -> int:
3 |     if b:
  |        - ... if this expression is `False`

Guppy compilation failed due to 1 previous error


Similarly, variables must have a unique type when they are used.

### Different types

In [3]:

@guppy
def bad(b: bool) -> int:
    if b:
        x = 4 


    else:
        x = True
    return int(x)  # x has different types depending on b

bad.compile()  # Raises an error

Error: Different types (at <In[3]>:9:15)
  | 
7 |     else:
8 |         x = True
9 |     return int(x)  # x has different types depending on b
  |                ^ Variable `x` may refer to different types

Notes:
  | 
3 |     if b:
4 |         x = 4
  |         - This is of type `int`

7 |     else:
8 |         x = True
  |         - This is of type `bool`

Guppy compilation failed due to 1 previous error


In [4]:

@guppy
def bad(b: bool) -> int:
    if b:
        x = 4 

    else:
        x = True
    return int(x)  # x has different types depending on b

bad.compile()  # Raises an error

Error: Different types (at <In[4]>:8:15)
  | 
6 |     else:
7 |         x = True
8 |     return int(x)  # x has different types depending on b
  |                ^ Variable `x` may refer to different types

Notes:
  | 
3 |     if b:
4 |         x = 4
  |         - This is of type `int`

6 |     else:
7 |         x = True
  |         - This is of type `bool`

Guppy compilation failed due to 1 previous error


In [5]:

@guppy
def bad(b: bool) -> int:
    if b:
        x = 4 
    else:
        x = True
    return int(x)  # x has different types depending on b

bad.compile()  # Raises an error

Error: Different types (at <In[5]>:7:15)
  | 
5 |     else:
6 |         x = True
7 |     return int(x)  # x has different types depending on b
  |                ^ Variable `x` may refer to different types

Notes:
  | 
3 |     if b:
4 |         x = 4
  |         - This is of type `int`
5 |     else:
6 |         x = True
  |         - This is of type `bool`

Guppy compilation failed due to 1 previous error


In [6]:

@guppy
def bad(b: bool) -> int:
    x = 4 if b else True
    return int(x)  # x has different types depending on b

bad.compile()  # Raises an error

Error: Different types (at <In[6]>:3:8)
  | 
1 | @guppy
2 | def bad(b: bool) -> int:
3 |     x = 4 if b else True
  |         ^^^^^^^^^^^^^^^^ Expression may refer to different types

Notes:
  | 
2 | def bad(b: bool) -> int:
3 |     x = 4 if b else True
  |         - This is of type `int`
  |     x = 4 if b else True
  |                     ---- This is of type `bool`

Guppy compilation failed due to 1 previous error


In [7]:

@guppy
def foo(b: bool) -> int:
    if b:
        def bar() -> int:
            return 0
    else:
        def bar() -> bool:
            return False

    return bar()

foo.compile_function()

Error: Different types (at <In[7]>:10:11)
   | 
 8 |             return False
 9 | 
10 |     return bar()
   |            ^^^ Variable `bar` may refer to different types

Notes:
   | 
 3 |     if b:
 4 |         def bar() -> int:
   |         -----------------
 5 |             return 0
   | -------------------- This is of type `() -> int`
 6 |     else:
 7 |         def bar() -> bool:
   |         ------------------
 8 |             return False
   | ------------------------ This is of type `() -> bool`

Guppy compilation failed due to 1 previous error


In [8]:

@guppy
def foo(b: bool) -> int:
    if b:
        def bar() -> int:
            return 0
    else:


        
        def bar() -> bool: 
            return False

    return bar()

foo.compile_function()

Error: Different types (at <In[8]>:13:11)
   | 
11 |             return False
12 | 
13 |     return bar()
   |            ^^^ Variable `bar` may refer to different types

Notes:
   | 
 3 |     if b:
 4 |         def bar() -> int:
   |         -----------------
 5 |             return 0
   | -------------------- This is of type `() -> int`

 9 | 
10 |         def bar() -> bool:
   |         ------------------
11 |             return False
   | ------------------------ This is of type `() -> bool`

Guppy compilation failed due to 1 previous error


### Copy violation

In [9]:
@guppy
def bad(q: qubit @owned) -> qubit:
    cx(q, q)
    return q

bad.compile()  # Raises an error

Error: Copy violation (at <In[9]>:3:10)
  | 
1 | @guppy
2 | def bad(q: qubit @owned) -> qubit:
3 |     cx(q, q)
  |           ^ Variable `q` with non-copyable type `qubit` cannot be
  |             borrowed ...

Note:
  | 
2 | def bad(q: qubit @owned) -> qubit:
3 |     cx(q, q)
  |        - Variable `q` borrowed here

Guppy compilation failed due to 1 previous error


In [10]:
@guppy.declare
def foo(q1: qubit) -> None: ...


@guppy.declare
def use(q: qubit @owned) -> None: ...


@guppy
def test(q: qubit @owned) -> None:
    use(q)


    
    foo(q)


test.compile()


Error: Copy violation (at <In[10]>:15:8)
   | 
13 | 
14 | 
15 |     foo(q)
   |         ^ Variable `q` with non-copyable type `qubit` cannot be
   |           borrowed ...

Note:
   | 
10 | def test(q: qubit @owned) -> None:
11 |     use(q)
   |         - Variable `q` consumed here

Guppy compilation failed due to 1 previous error


In [11]:
@guppy.declare
def foo(q1: qubit) -> None: ...


@guppy.declare
def use(q: qubit @owned) -> None: ...


@guppy
def test(q: qubit @owned) -> None:
    use(q)
    foo(q)


test.compile()


Error: Copy violation (at <In[11]>:12:8)
   | 
10 | def test(q: qubit @owned) -> None:
11 |     use(q)
12 |     foo(q)
   |         ^ Variable `q` with non-copyable type `qubit` cannot be
   |           borrowed ...

Note:
   | 
10 | def test(q: qubit @owned) -> None:
11 |     use(q)
   |         - Variable `q` consumed here

Guppy compilation failed due to 1 previous error


In [12]:
@guppy.struct
class MyStruct:
    q1: qubit
    q2: qubit


@guppy
def foo(b: bool, s: MyStruct @owned) -> MyStruct:
    if b:
        measure(s.q2)


        
    return s


foo.compile()


Error: Copy violation (at <In[12]>:14:11)
   | 
12 | 
13 | 
14 |     return s
   |            ^ Field `s.q2` with non-copyable type `qubit` cannot be
   |              returned ...

Note:
   | 
 9 |     if b:
10 |         measure(s.q2)
   |                 ---- Field `s.q2` consumed here

Guppy compilation failed due to 1 previous error


In [13]:
@guppy
def main() -> None:
   xs = array(1, 2, 3)
   ys = xs
   xs[0] = 0


main.compile()

Error: Copy violation (at <In[13]>:5:3)
  | 
3 |    xs = array(1, 2, 3)
4 |    ys = xs
5 |    xs[0] = 0
  |    ^^^^^ Variable `xs` with non-copyable type `array[int, 3]` cannot
  |          be borrowed ...

Note:
  | 
3 |    xs = array(1, 2, 3)
4 |    ys = xs
  |         -- Variable `xs` moved here

Help: Consider copying `xs` instead of moving it: `xs.copy()`

Guppy compilation failed due to 1 previous error


In [14]:
@guppy.struct
class MyStruct:
    q: qubit


@guppy
def foo(s: MyStruct @owned) -> tuple[qubit, MyStruct]:
    return s.q, s


foo.compile()

Error: Copy violation (at <In[14]>:8:16)
  | 
6 | @guppy
7 | def foo(s: MyStruct @owned) -> tuple[qubit, MyStruct]:
8 |     return s.q, s
  |                 ^ Field `s.q` with non-copyable type `qubit` cannot be
  |                   returned ...

Note:
  | 
7 | def foo(s: MyStruct @owned) -> tuple[qubit, MyStruct]:
8 |     return s.q, s
  |            --- Field `s.q` returned here

Guppy compilation failed due to 1 previous error


In [15]:
@guppy.declare
def foo(q1: qubit) -> None: ...


@guppy.declare
def use(q: qubit @owned) -> None: ...


@guppy
def test(q: qubit @owned) -> None:
    use(q)
    foo(q)


test.compile()

Error: Copy violation (at <In[15]>:12:8)
   | 
10 | def test(q: qubit @owned) -> None:
11 |     use(q)
12 |     foo(q)
   |         ^ Variable `q` with non-copyable type `qubit` cannot be
   |           borrowed ...

Note:
   | 
10 | def test(q: qubit @owned) -> None:
11 |     use(q)
   |         - Variable `q` consumed here

Guppy compilation failed due to 1 previous error


### Drop violation

In [16]:
@guppy
def bad(q: qubit @owned) -> qubit:
    tmp = qubit()
    cx(tmp, q)
    #discard(tmp)  # Compiler complains if tmp is not explicitly discarded
    return q

bad.compile()  # Raises an error

Error: Drop violation (at <In[16]>:3:4)
  | 
1 | @guppy
2 | def bad(q: qubit @owned) -> qubit:
3 |     tmp = qubit()
  |     ^^^ Variable `tmp` with non-droppable type `qubit` is leaked

Help: Make sure that `tmp` is consumed or returned to avoid the leak

Guppy compilation failed due to 1 previous error


In [17]:
@guppy.struct
class MyStruct:
    q: qubit


@guppy
def foo(b: bool) -> bool:
    s = MyStruct(qubit())
    if b:
        return measure(s.q)
    return False


foo.compile()

Error: Drop violation (at <In[17]>:8:4)
  | 
6 | @guppy
7 | def foo(b: bool) -> bool:
8 |     s = MyStruct(qubit())
  |     ^ Field `s.q` with non-droppable type `qubit` may be leaked
  |       ...

Note:
  | 
8 |     s = MyStruct(qubit())
9 |     if b:
  |        - if this expression is `False`

Help: Make sure that `s.q` is consumed or returned to avoid the leak

Guppy compilation failed due to 1 previous error


### Illegal assignment

In [18]:
@guppy
def outer(x: int) -> int:
    def nested() -> None:
        x += 1  # Mutation of captured variable x is not allowed
    nested()
    return x

outer.compile_function()  # Raises an error

Error: Illegal assignment (at <In[18]>:4:8)
  | 
2 | def outer(x: int) -> int:
3 |     def nested() -> None:
4 |         x += 1  # Mutation of captured variable x is not allowed
  |         ^^^^^^ Variable `x` may not be assigned to since `x` is captured
  |                from an outer scope

Note:
  | 
1 | @guppy
2 | def outer(x: int) -> int:
  |           ------ `x` defined here

Guppy compilation failed due to 1 previous error


In [19]:
@guppy.struct
class MyStruct1:
    x: "MyStruct2"


@guppy.struct
class MyStruct2:
    q: qubit


@guppy.declare
def use(s: MyStruct2 @owned) -> None: ...


@guppy
def foo(s: MyStruct1 @owned) -> qubit:
    use(s.x)
    return s.x.q


foo.compile()


Error: Copy violation (at <In[19]>:18:11)
   | 
16 | def foo(s: MyStruct1 @owned) -> qubit:
17 |     use(s.x)
18 |     return s.x.q
   |            ^^^^^ Field `s.x.q` with non-copyable type `qubit` cannot be
   |                  returned ...

Note:
   | 
16 | def foo(s: MyStruct1 @owned) -> qubit:
17 |     use(s.x)
   |         --- Field `s.x.q` consumed here

Guppy compilation failed due to 1 previous error


In [20]:
@guppy
def foo(qs: list[tuple[qubit, bool]] @owned) -> list[qubit]:
    rs: list[qubit] = []
    for q, b in qs:
        rs += [q]
        if b:
            break
    return rs


foo.compile()

Error: Break in non-droppable loop (at <In[20]>:7:12)
  | 
5 |         rs += [q]
6 |         if b:
7 |             break
  |             ^^^^^ Early exit in non-droppable loops is not allowed

Note:
  | 
3 |     rs: list[qubit] = []
4 |     for q, b in qs:
  |                 -- Iterator has non-droppable type `list[(qubit, bool)]`

Guppy compilation failed due to 1 previous error


In [21]:
@guppy.struct
class MyStruct:
    q: qubit


@guppy.declare
def use(q: qubit @owned) -> None: ...


@guppy
def test(s: MyStruct, b: bool) -> None:
    if b:
        use(s.q)


test.compile()

Error: Copy violation (at <In[21]>:11:9)
   | 
 9 | 
10 | @guppy
11 | def test(s: MyStruct, b: bool) -> None:
   |          ^^^^^^^^^^^ Borrowed argument s cannot be returned to the caller ...

Note:
   | 
12 |     if b:
13 |         use(s.q)
   |             --- since `s.q` with non-copyable type `qubit` was already
   |                 consumed here

Help: Consider writing a value back into `s.q` before returning

Guppy compilation failed due to 1 previous error


In [22]:



@guppy
def foo(x: int) -> int:
    y = x + 1

    def bar() -> None:

        def baz() -> None:
            y += 2

        baz()

    bar()
    return y

foo.compile_function()


Error: Illegal assignment (at <In[22]>:8:12)
  | 
6 | 
7 |         def baz() -> None:
8 |             y += 2
  |             ^^^^^^ Variable `y` may not be assigned to since `y` is captured
  |                    from an outer scope

Note:
  | 
2 | def foo(x: int) -> int:
3 |     y = x + 1
  |     - `y` defined here

Guppy compilation failed due to 1 previous error


In [23]:
@guppy
def foo(x: int) -> int:
    y = x + 1

    def bar() -> None:
        y += 2

    bar()
    return y

foo.compile_function()

Error: Illegal assignment (at <In[23]>:6:8)
  | 
4 | 
5 |     def bar() -> None:
6 |         y += 2
  |         ^^^^^^ Variable `y` may not be assigned to since `y` is captured
  |                from an outer scope

Note:
  | 
2 | def foo(x: int) -> int:
3 |     y = x + 1
  |     - `y` defined here

Guppy compilation failed due to 1 previous error


### Others

In [24]:
@guppy
def test() -> None:
    with dagger:
        for _ in range(46):
            pass


test.compile()

Error: Invalid expression in dagger (at <In[24]>:4:8)
  | 
2 | def test() -> None:
3 |     with dagger:
4 |         for _ in range(46):
  |         ^^^^^^^^^^^^^^^^^^^
5 |             pass
  | ^^^^^^^^^^^^^^^^ Loop found in a dagger context

Note:
  | 
2 | def test() -> None:
3 |     with dagger:
  |          ------ dagger modifier is used here

Guppy compilation failed due to 1 previous error
